**** データの入手・整形 ****

文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
上述1と2の内容を結合（concatenate）し，行をランダムに並び替える

sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [1]:
# coding: utf-8
import codecs
import random

fname_pos = 'nlp100data/rt-polaritydata/rt-polarity.pos'
fname_neg = 'nlp100data/rt-polaritydata/rt-polarity.neg'
fname_smt = 'nlp100data/sentiment.txt'
fencoding = 'cp1252'        # Windows-1252らしい

In [ ]:
result = []

ポジティブデータの読み込み

In [ ]:
with codecs.open(fname_pos, 'r', fencoding) as file_pos:
    result.extend(['+1 {}'.format(line.strip()) for line in file_pos])

In [ ]:
print("number of postive sentences is {}".format(len(result)))

In [ ]:
result_neg = []
with codecs.open(fname_neg, 'r', fencoding) as file_neg:
    result_neg.extend(['-1 {}'.format(line.strip()) for line in file_neg])


In [ ]:
print("number of negative sentences is {}".format(len(result_neg)))

In [ ]:
result.extend(result_neg)

In [ ]:
print("total number of sentences is {}".format(len(result)))

シャッフル

In [ ]:
random.shuffle(result)

書き出し

In [ ]:
with codecs.open(fname_smt, 'w', fencoding) as file_out:
    print(*result, sep='\n', file=file_out)

####  ストップワード

英語のストップワードのリスト（ストップリスト）を適当に作成せよ．

さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．

さらに，その関数に対するテストを記述せよ．

ストップワードのリスト  http://xpo6.com/list-of-english-stop-words/ のCSV Formatより

In [2]:
stop_words = (
    'a,able,about,across,after,all,almost,also,am,among,an,and,any,are,'
    'as,at,be,because,been,but,by,can,cannot,could,dear,did,do,does,'
    'either,else,ever,every,for,from,get,got,had,has,have,he,her,hers,'
    'him,his,how,however,i,if,in,into,is,it,its,just,least,let,like,'
    'likely,may,me,might,most,must,my,neither,no,nor,not,of,off,often,'
    'on,only,or,other,our,own,rather,said,say,says,she,should,since,so,'
    'some,than,that,the,their,them,then,there,these,they,this,tis,to,too,'
    'twas,us,wants,was,we,were,what,when,where,which,while,who,whom,why,'
    'will,with,would,yet,you,your').lower().split(',')

In [3]:
def is_stopword(str):
    '''文字がストップワードかどうかを返す
    大小文字は同一視する

    戻り値：
    ストップワードならTrue、違う場合はFalse
    '''
    return str.lower() in stop_words


In [4]:
assert is_stopword("Tokyo")

AssertionError: 

In [5]:
assert is_stopword("a")

#### 素性抽出

極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．

素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

**** coreNLPを使ったほうがよさそう。nlp100_50.ipynbの先頭を踏襲すればよい（？）****

https://github.com/Lynten/stanford-corenlp

In [ ]:
with codecs.open(fname_smt, 'r', fencoding) as rf:
    for i , l in enumerate(rf):
        for w in l[3:].split(" "):
            print(w.strip())
            
            
        

In [7]:
from stanfordcorenlp import StanfordCoreNLP

In [8]:
corenlp = StanfordCoreNLP(r'/home/toshinao/Tools/stanford-corenlp-full-2018-02-27')

In [9]:
from bs4 import BeautifulSoup

In [10]:
props={'annotators': 'lemma','pipelineLanguage':'en','outputFormat':'xml'}

In [135]:
xmls = list()
with codecs.open(fname_smt, 'r', fencoding) as rf:
    for l in rf:
        run_xml = corenlp.annotate(l[3:] , properties=props)
        xmls.append(run_xml)
            
        

In [ ]:
len(xmls)

In [ ]:
"x" not in stop_words

In [138]:
from collections import Counter

In [139]:
word_counter = Counter()

In [140]:
for x in xmls:
    soup = BeautifulSoup(x, 'lxml')
    for x in soup.find_all("lemma"):
        y = x.get_text()
        if y not in stop_words:
            word_counter.update([y])
            

In [141]:
features = [word for word, count in word_counter.items() if count >= 6]

In [142]:
print("number of seletcted features is {}".format(len(features)))

number of seletcted features is 3439


In [143]:
features[0:10]

['play',
 'droll',
 'hear',
 'worry',
 'promise',
 'conclusion',
 'nonsense',
 'pumpkin',
 'minor',
 'unintentionally']

In [16]:
fname_features = "nlp100data/features.txt"

In [144]:
with codecs.open(fname_features, 'w', fencoding) as file_out:
    print(*features, sep='\n', file=file_out)

In [145]:
def load_dict_features():
    '''features.txtを読み込み、素性をインデックスに変換するための辞書を作成
    インデックスの値は1ベースで、features.txtにおける行番号と一致する。

    戻り値：
    素性をインデックスに変換する辞書
    '''
    with codecs.open(fname_features, 'r', fencoding) as file_in:
        return [l.strip() for l in file_in]

##### sandbox

In [ ]:
fname_features

In [ ]:
print(soup.prettify())

In [ ]:
%%bash
git commit -a -m "from notebook"

In [ ]:
features = set()

for x in xmls:
    soup = BeautifulSoup(x, 'lxml')
    for x in soup.find_all("lemma"):
        y = x.get_text()
        if y not in stop_words:
            features.add(y)
            

In [ ]:
for x in soup.find_all("lemma"):
    print(x.get_text())

In [ ]:
run_xml = corenlp.annotate(l, properties=props)

In [ ]:
corenlp.parse(l)

https://github.com/Lynten/stanford-corenlp

In [ ]:
props={'annotators': 'tokenize,ssplit,pos','pipelineLanguage':'en','outputFormat':'xml'}
print(corenlp.annotate(l, properties=props))

#### 学習
72で抽出した素性を用いて，ロジスティック回帰モデルを学習せよ．

In [146]:
import numpy as np

In [147]:
from sklearn.feature_extraction.text import CountVectorizer as CV

In [148]:
features = load_dict_features()

In [149]:
cv = CV(vocabulary=features)

In [150]:
cv.transform(features[0:3])


<3x3439 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [151]:
np.sum(cv.transform(["shibuya"]).toarray())

0

In [152]:
token_sum_list = []
for x in xmls:
    soup = BeautifulSoup(x, 'lxml')
    lemmas = list()
    for x in soup.find_all("lemma"):
        xt = x.get_text()
        if xt not in stop_words:
            lemmas.append(xt)
    token_matrix = cv.transform(lemmas).toarray()
    token_sum = token_matrix.sum(axis = 0).astype(float)
    token_sum_list.append(token_sum.tolist())

In [153]:
token_sum_matrix = np.array(token_sum_list)

In [154]:
token_sum_matrix.shape

(10662, 3439)

In [156]:
with codecs.open(fname_smt, 'r', fencoding) as rf:
    y = [float(l[0:2]) for l in rf ]

In [157]:
y = np.array(y)

In [158]:
y.shape

(10662,)

##### 回帰

http://ailaby.com/logistic_reg/

In [159]:
from sklearn.linear_model import LogisticRegression
 

In [160]:
lr = LogisticRegression()

In [161]:
lr.fit(X = token_sum_matrix , y = y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [163]:
lr.score(X=token_sum_matrix , y = y)

0.87225661226786722

#### 予測

73で学習したロジスティック回帰モデルを用い，与えられた文の極性ラベル（正例なら"+1"，負例なら"-1"）と，その予測確率を計算するプログラムを実装せよ．

In [190]:
y_pred1 = lr.predict(X = token_sum_matrix)

In [169]:
y[0:10]

array([-1., -1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.])

In [172]:
lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [204]:
lr.coef_

array([[-0.35716411,  0.550107  , -0.22356707, ..., -0.38271609,
        -0.82514424,  0.50568172]])

In [174]:
lr.intercept_

array([-0.27312972])

In [198]:
fy_pred2 = 1/ ( 1 + np.exp(-lr.intercept_ -token_sum_matrix.dot(np.transpose(lr.coef_))))

In [199]:
y_pred2 = [1 if x > 0.5 else -1 for x in fy_pred2]

In [200]:
[(y_pred1[i] , fy_pred2[i]) for i in range(len(y_pred2)) if y_pred2[i] != y_pred1[i]]

[]

#####  sandbox

In [ ]:
lr.fit?


In [50]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler

In [54]:
np.random.seed(seed=0)
X_0 = np.random.multivariate_normal( [2,2],  [[2,0],[0,2]],  50 )
y_0 = np.zeros(len(X_0))
 
X_1 = np.random.multivariate_normal( [6,7],  [[3,0],[0,3]],  50 )
y_1 = np.ones(len(X_1))
 
X = np.vstack((X_0, X_1))
ytmp = np.append(y_0, y_1)
 
X_train, X_test, y_train, y_test = train_test_split(X, ytmp, test_size=0.3)
 
# 特徴データを標準化(平均 0、標準偏差 1)
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

https://note.nkmk.me/python-numpy-eye-identity-one-hot/

In [ ]:
np.eye(10)[[1,2]]

In [ ]:
tmp = np.empty()

In [ ]:
x = xmls[1]

In [ ]:
soup = BeautifulSoup(x, 'lxml')
lemmas = list()
for x in soup.find_all("lemma"):
    y = x.get_text()
    if y not in stop_words:
        lemmas.append(y)

In [ ]:
tmp = cv.transform(lemmas).toarray()

In [ ]:
tmp.sum(axis = 0).tolist()

In [211]:
1+1

2

In [208]:
%%bash 
git commit -a -m"from notebook"

On branch master
Your branch is ahead of 'origin/master' by 3 commits.
  (use "git push" to publish your local commits)
Untracked files:
	.idea/
	.ipynb_checkpoints/
	error.txt
	nlp.txt.xml
	nlp100data/
	nlp_parsed.xml
	predicate.txt
	result.txt
	results/

nothing added to commit but untracked files present


In [207]:
%%bash 
git push

Git 2.0 from 'matching' to 'simple'. To squelch this message
and maintain the traditional behavior, use:

  git config --global push.default matching

To squelch this message and adopt the new behavior now, use:

  git config --global push.default simple

When push.default is set to 'matching', git will push local branches
to the remote branches that already exist with the same name.

Since Git 2.0, Git defaults to the more conservative 'simple'
behavior, which only pushes the current branch to the corresponding
remote branch that 'git pull' uses to update the current branch.

See 'git help config' and search for 'push.default' for further information.
(the 'simple' mode was introduced in Git 1.7.11. Use the similar mode
'current' instead of 'simple' if you sometimes use older versions of Git)

fatal: could not read Username for 'https://github.com': No such device or address
